Q1.3. Which are the top 10 pizza restaurants by rating?

In [1]:
# Import required libraries
import sqlite3
import pandas as pd

databases = ['../databases/deliveroo.db', '../databases/takeaway.db', '../databases/ubereats.db']

# Connect to the first database
conn = sqlite3.connect(databases[0])
cursor = conn.cursor()

# Dynamically attach other databases
for i, db in enumerate(databases[1:], start=2):
    cursor.execute(f"ATTACH DATABASE '{db}' AS db{i}")

In [2]:
query1 = """SELECT DISTINCT id, name, rating FROM (SELECT DISTINCT id, name, category, rating 
            FROM restaurants 
            WHERE category LIKE '%Pizza%' OR
            category LIKE '%pizza%'
             
            UNION
            SELECT DISTINCT restaurants.id AS id, restaurants.title AS name,  categories.category, restaurants.rating__rating_value AS rating
            FROM db3.restaurants 
            JOIN db3.restaurant_to_categories AS categories ON restaurants.id == categories.restaurant_id
            WHERE categories.category LIKE '%Pizza%' OR categories.category LIKE '%pizza%'

            UNION
            SELECT DISTINCT restaurants.primarySlug AS id, restaurants.name, categories.name, restaurants.ratings AS rating 
            FROM db2.categories
            JOIN db2.categories_restaurants ON categories.restaurant_id == categories_restaurants.restaurant_id
            JOIN db2.restaurants ON categories.restaurant_id == restaurants.primarySlug
            WHERE categories.name LIKE '%Pizza%' OR
            categories.name LIKE '%pizza%')
            ORDER BY rating DESC LIMIT 10
            ;
            """

df_pizza = pd.read_sql_query(query1, conn)
df_pizza#[['name', 'rating']]

,id,name,rating
0,eethuis-carlos-aarschot,De Echte Eethuis Carlos,5.0
1,kebap-pizza-de-brug,Kebap - Pizza De Brug,5.0
2,la-trca,La Trinacria Bottega,5.0
3,munchies-ieper,Munchies Ieper,5.0
4,snack-it,Snack-It,5.0
5,trend-kebab-pizzeria,Trend Kebab & Pizzeria,5.0
6,17478ff5-ba5d-5166-ba01-48d1f97cb588,American pizza,4.9
7,5a603fd8-4bfa-44e7-a4ad-47063893957f,Fratello's,4.9
8,am-princesse,Am Princesse,4.9
9,b0bc113f-86e9-41ed-95e1-7e8d4957ac9f,Pizza Autentica,4.9


In [5]:
import plotly.express as px
fig = px.bar(df_pizza, x='name', y="rating",
               labels={"name": "Name of restaurants serving pizza", "rating": "Rating for the restaurant"}, 
              title = "Top 10 pizza restaurants by rating")
fig.show()

Q2.3. Where are the delivery ‘dead zones’—areas with minimal restaurant coverage?

In [77]:
query2 = """SELECT count_of_restaurant, postal_code, latitude, longitude FROM (SELECT DISTINCT COUNT(name) AS count_of_restaurant, postal_code,
AVG(latitude) AS latitude, AVG(longitude) AS longitude
FROM restaurants
WHERE postal_code IS NOT NULL
GROUP BY postal_code
UNION

SELECT DISTINCT COUNT(restaurants.name) AS count_of_restaurant, locations.postalCode AS postal_code,
AVG(locations.latitude) AS latitude, AVG(locations.longitude) AS longitude
FROM db2.restaurants
JOIN db2.locations_to_restaurants ON restaurants.primarySlug == locations_to_restaurants.restaurant_id
JOIN db2.locations ON locations_to_restaurants.location_id == locations.ID
WHERE postal_code IS NOT NULL
GROUP BY postal_code
UNION
SELECT DISTINCT COUNT(restaurants.title) AS count_of_restaurant,
       restaurants.location__postal_code as postal_code,
       AVG(restaurants.location__latitude) as latitude, 
       AVG(restaurants.location__longitude) as longitude
FROM db3.restaurants
WHERE postal_code IS NOT NULL
GROUP BY postal_code)
WHERE count_of_restaurant < 265
ORDER BY count_of_restaurant

"""

df_deadzone = pd.read_sql_query(query2, conn)
df_deadzone.to_csv('dead_zone.csv')

In [78]:
from pandas.api.types import is_numeric_dtype
import numpy as np
def check_type_numeric_cols(df):
    '''
    Checks if all numeric columns are really numeric and changes them otherwise.
    It takes and returns a df.
    '''   
    numeric_columns_df =  df[["latitude", "longitude"]]


    # check the selected columns are numeric same as the one specified
    numeric_columns = df.select_dtypes(include=np.number).columns
    is_all_numeric = len(numeric_columns) == len(numeric_columns_df.columns)
    if(not is_all_numeric):
        convert_dict = { }
        for column in numeric_columns_df:
            # Select column contents by column
            columnSeriesObj = numeric_columns_df[column]
            if is_numeric_dtype(columnSeriesObj.dtype):
                convert_dict[column] = columnSeriesObj.dtype
            else:
                convert_dict[column] = np.float64
        df = df.astype(convert_dict)
    
    return df

In [79]:
average = df_deadzone['count_of_restaurant'].mean()
print(average)

65.44190871369294


In [80]:
df_deadzone = check_type_numeric_cols(df_deadzone)
df = df_deadzone.dropna(subset=['latitude', 'longitude'])

import folium
from folium.plugins import HeatMap
from matplotlib import colormaps
from matplotlib.colors import to_hex

# Create a base map
#m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=10)
m = folium.Map(location=[50.8503, 4.3517], zoom_start=8)  # Centered on Belgium

min_count = df['count_of_restaurant'].min()
max_count = df['count_of_restaurant'].max()

df['color'] = df['count_of_restaurant'].apply(lambda x: (x - min_count) / (max_count - min_count))
#df['radius'] = df['count_of_restaurant'].apply(lambda x: (max_count - x) / (max_count - min_count) * 15 + 5)
# Add markers to the map
cmap = colormaps['RdYlGn'].reversed()
for _, row in df.iterrows():
    color = 'red' #if row['count_of_restaurant'] == 0 else 'green'
    color = to_hex(cmap(row['color']))  # Convert RGBA to HEX
    folium.CircleMarker(
        location=(row['latitude'], row['longitude']),
        radius= 3, #row['radius'], #+ 3,  # Add small radius for visibility
        color=color,
        fill=True,
        fill_opacity=0.7,
        popup=f"postal_code: {row['postal_code']}<br>count: {row['count_of_restaurant']}"
    ).add_to(m)

# Save or display the map
m.save('dead_zones_map.html')
m

Q2.6. Identify the World Hummus Order (WHO); top 3 hummus serving restaurants.

In [7]:
query3 = """SELECT DISTINCT id, name, category, rating 
            FROM restaurants 
            WHERE category LIKE '%Hummus%' OR
            category LIKE '%hommus%'
             
            UNION
            SELECT DISTINCT restaurants.id AS id, restaurants.title AS name,  categories.category, restaurants.rating__rating_value AS rating
            FROM db3.restaurants 
            JOIN db3.restaurant_to_categories AS categories ON restaurants.id == categories.restaurant_id
            WHERE categories.category LIKE '%Hummus%' OR categories.category LIKE '%hommus%'

            UNION
            SELECT DISTINCT restaurants.primarySlug AS id, restaurants.name, categories.name, restaurants.ratings AS rating 
            FROM db2.categories
            JOIN db2.categories_restaurants ON categories.restaurant_id == categories_restaurants.restaurant_id
            JOIN db2.restaurants ON categories.restaurant_id == restaurants.primarySlug
            WHERE categories.name LIKE '%Hummus%' OR
            categories.name LIKE '%hommus%'
            ORDER BY rating DESC LIMIT 10
            ;
            """

df_hummus = pd.read_sql_query(query3, conn)
df_hummus[['name', 'rating']]

,name,rating


In [ ]:
# Detach databases
for i in range(2, len(databases) + 1):
    cursor.execute(f"DETACH DATABASE db{i}")

# Close connection
conn.close()